In [1]:
from zvsn_utils import *

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Reading images from folder and preparing dataset

In [3]:
img_width, img_height = 150, 150
X,y = prepare_data("../../dataset/Normal_vs_Zoomed", 150, 150)

Getting the images from the data folder
2 classes were found:['Normal' 'Zoomed']
Total number of images in the dataset: 2041
Dimensions of images in the dataset: (150, 150, 3)


In [4]:
X[:1].shape

(1, 150, 150, 3)

### Splitting the dataset into train, test and validation sets

In [5]:
from sklearn.model_selection import train_test_split

#Splitting 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=33)
print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))

Train set size: 1632, Val set size: 205, Test set size: 204


### Image Preprocessing

In [8]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [9]:
datagen.fit(X_train)

### Creating a preview folder to view preprocessed images 

In [ ]:
!rm -r preview
!mkdir preview

i = 0
for batch in datagen.flow(dataset[:1], batch_size=1,
                          save_to_dir='preview', save_prefix='test', save_format='jpeg'):
    i += 1
    if i > 5:
        break  # otherwise the generator would loop indefinitely

!ls preview/

In [ ]:
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image

modified_images = glob.glob('preview/**/*jpeg')
for img in modified_images:
    display(_Imgdis(filename=img))

### Model Training

In [ ]:
# get model definition
model = zvsn_model()

# train model
model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),
                    steps_per_epoch=len(X_train) / 128, validation_data=(X_val,y_val), epochs=10,verbose=2)

In [ ]:
# save model
model.save('zoomed_vs_normal.h5')

### Loading the model and applying it on test data

In [2]:
from keras.models import load_model
model = load_model('zoomed_vs_normal.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_2 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_3 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [10]:
# get predictions for test set
y_test_pred = predict(model, datagen, X_test)

In [14]:
# get classification metrics 
print_metrics(y_test, y_test_pred)

             precision    recall  f1-score   support

          0       0.95      0.86      0.90        71
          1       0.93      0.98      0.95       133

avg / total       0.94      0.94      0.94       204



In [15]:
datagen.mean, datagen.std

(array([[[ 116.96533966,  116.96533966,  116.96533966]]], dtype=float32),
 array([[[ 79.82288361,  79.79962921,  75.93218994]]], dtype=float32))

### Predict on other dataset

In [ ]:
# load model
from keras.models import load_model
model = load_model('zoomed_vs_normal.h5')

In [ ]:
import numpy as np
def preprocess(X):
    mean = np.array([27.04171, 27.04171, 27.04171]).reshape(1,1,3)
    std = np.array([83.20263, 83.20263, 83.20263]).reshape(1,1,3)
    X -= mean
    X /= (std + K.epsilon())
    return X    

In [ ]:
# preprocess images
from keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(preprocessing_function=preprocess)
sample_images = test_datagen.flow_from_directory("../../dataset/Normal_vs_Zoomed/Normal/",
                                              target_size=(150,150),
                                              class_mode=None,
                                              shuffle=False)

In [ ]:
# get predictions
%%time
predictions = model.predict_generator(sample_images)

In [ ]:
# transform predictions into a dataframe
import pandas as pd
pred_df = pd.DataFrame({'filenames':cargo_test.filenames,'predictions':predictions.ravel()})
pred_df['label'] = (pred_df.predictions > 0.5).astype(int)
pd.options.display.max_colwidth=-1
pred_df.head()

In [ ]:
# view images that are predicted as zoomed by the model
for img in pred_df[pred_df.label==1].sample(frac=1.0).head().iterrows():
    display(_Imgdis(filename='../datasets/Normal_vs_Zoomed/test/'+img[1].filenames))

In [ ]:
# view images that are predicted as normal by the model
for img in pred_df[pred_df.label==0].sample(frac=1.0).head().iterrows():
    display(_Imgdis(filename='../datasets/Normal_vs_Zoomed/test/'+img[1].filenames))